In [54]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import keras
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder 
from sklearn.cross_validation import train_test_split

In [55]:
#import data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [56]:
def prep_data(data, train=True):
    
    #if training data...
    if train:
        
        #shuffle data
        data = data.reindex(np.random.permutation(data.index))
        
        #prepare feature/target datasets
        X = data.drop(['id','target'],axis=1)
        y = data['target']
        
        #convert type
        X = X.astype('float')
        
        #standardize feature set
        scaler = StandardScaler()
        scaler.fit(X)
        X = scaler.transform(X)
        
        #encode classes as ints
        encoder = LabelEncoder()
        encoder.fit(y)
        y = encoder.transform(y).astype(np.int32)
        
        #convert target vector to categorical matrix
        y = np_utils.to_categorical(y) 
        
        return [X,y]
        
    #if testing data...    
    else:
        
        #drop 'id' field
        X = data.drop(['id'],axis=1)
        
        #convert type
        X = X.astype('float')
        
        #standardize feature set
        scaler = StandardScaler()
        scaler.fit(X)
        X = scaler.transform(X)
        
        return X

In [57]:
training_data = train.copy()

X, y = prep_data(training_data,train=True)

In [58]:
dims = X.shape[1]
print(dims, 'dimensions')

nb_classes = len(y[0])
print(nb_classes, 'classes')

93 dimensions
9 classes


In [59]:
#prep for model
fBestModel = 'best_model.h5' 
early_stop = EarlyStopping(monitor='val_loss', patience=4, verbose=1) 
best_model = ModelCheckpoint(fBestModel, verbose=0, save_best_only=True)

In [60]:
def baseline_model():
    #create model
    model = Sequential()
    model.add(Dense(100, input_shape=(dims,)))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    model.compile(optimizer='sgd', loss='categorical_crossentropy')
    
    return model

In [65]:
def model():
    #create model
    model = Sequential()
    model.add(Dense(1000, input_shape=(dims,)))
    model.add(Dropout(0.05))
    model.add(Dense(1000, activation='relu'))
    model.add(Dropout(0.05))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.05))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    model.compile(optimizer='sgd', loss='categorical_crossentropy')
    
    return model

## Cross-Validating Model

In [66]:
#cross-validation
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [67]:
print('Building model...')
model = model()

print('Training cross-validated model...')
model.fit(x_train, y_train, epochs=100,
    batch_size=2000, verbose=True, validation_split=0.15, 
    callbacks=[best_model, early_stop])
print('CV Training Complete.')

Building model...
Training cross-validated model...
Train on 44706 samples, validate on 7890 samples
Epoch 1/100
44706/44706 [==============================] - 5s - loss: 1.9829 - val_loss: 1.7555
Epoch 2/100
44706/44706 [==============================] - 5s - loss: 1.6540 - val_loss: 1.5086
Epoch 3/100
44706/44706 [==============================] - 5s - loss: 1.4501 - val_loss: 1.3391
Epoch 4/100
44706/44706 [==============================] - 5s - loss: 1.3075 - val_loss: 1.2172
Epoch 5/100
44706/44706 [==============================] - 5s - loss: 1.2001 - val_loss: 1.1273
Epoch 6/100
44706/44706 [==============================] - 5s - loss: 1.1211 - val_loss: 1.0584
Epoch 7/100
44706/44706 [==============================] - 5s - loss: 1.0598 - val_loss: 1.0042
Epoch 8/100
44706/44706 [==============================] - 5s - loss: 1.0116 - val_loss: 0.9608
Epoch 9/100
44706/44706 [==============================] - 5s - loss: 0.9718 - val_loss: 0.9252
Epoch 10/100
44706/44706 [=========

In [68]:
print('Evaluating model...')
model.evaluate(x_test,y_test, batch_size=100)

Evaluating model...
8900/9282 [===========================>..] - ETA: 0s

0.590264311460451

## Test Predictions

In [69]:
test_data = test.copy()

X_test = prep_data(test_data,train=False)

In [70]:
predictions = model.predict_classes(X_test)

144000/144368 [============================>.] - ETA: 0s

In [71]:
results = np_utils.to_categorical(predictions)

In [72]:
df = pd.DataFrame(results)

In [73]:
rng = list(range(1,len(results)+1))
df['id']= rng
df = df.set_index('id')

In [74]:
cols = []

for i in range(1,10):
    cols.append("Class_"+str(i))
    
df.columns = cols

In [75]:
df.to_csv('results_7_19_2017_06_00_00.csv')